## Library

In [1]:
import numpy as np
import tensorflow as tf
from copy import deepcopy

from transformers import BertTokenizerFast
import random

import pickle
from tqdm.notebook import tqdm

import import_ipynb
from QBert.QDataWriter import *

def load_pkl(file_path) :
    
    with open(file_path, 'rb') as f:
        df = pickle.load(f)
        
    return df

def save_pkl(df, file_path) :
    
    with open(file_path, 'wb') as f:
        pickle.dump(df, f)
        

importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\QDataWriter.ipynb


In [11]:


file_path = ['./dt/wiki_ko_dict_sample.pkl']
output_files = ['./Test_Examples.tfrecords']

tokenizer_for_load = BertTokenizerFast.from_pretrained('./model/BertTokenizer-3000-32000-vocab.txt'
                                                   , strip_accents=False
                                                   , lowercase=False)

max_seq_length = 256
dupe_factor = 2
short_seq_prob = .15
masked_lm_prob = .15
max_predictions_per_seq = 255
seed = 719
rng = random.Random(seed)


instances = create_training_instances(input_files=file_path
                                    , tokenizer = tokenizer_for_load
                                    , max_seq_length = max_seq_length
                                    , dupe_factor = dupe_factor
                                    , short_seq_prob = short_seq_prob
                                    , masked_lm_prob = masked_lm_prob
                                    , max_predictions_per_seq = max_predictions_per_seq
                                    , rng = rng)

Calling BertTokenizerFast.from_pretrained() with the path to a single file or url is deprecated


In [12]:
write_instance_to_example_files(instances = instances
                                , tokenizer =  tokenizer_for_load
                                , max_seq_length = max_seq_length
                                , max_predictions_per_seq = max_predictions_per_seq
                                , output_files = output_files
                                , gzip_compress = False
                                , use_v2_feature_names = False)

In [13]:
filenames = ['./Test_Examples.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [14]:
# Create a description of the features.
feature_description = {
    'input_ids': tf.io.FixedLenFeature([256], tf.int64),
    'segment_ids': tf.io.FixedLenFeature([256], tf.int64),
    'input_mask': tf.io.FixedLenFeature([256], tf.int64),
    'masked_lm_positions': tf.io.FixedLenFeature([255], tf.int64),
    'masked_lm_ids': tf.io.FixedLenFeature([255], tf.int64),
    'masked_lm_weights': tf.io.FixedLenFeature([255], tf.float32),
    'next_sentence_labels': tf.io.FixedLenFeature([1], tf.int64),
}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

In [15]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

<MapDataset shapes: {input_ids: (256,), input_mask: (256,), masked_lm_ids: (255,), masked_lm_positions: (255,), masked_lm_weights: (255,), next_sentence_labels: (1,), segment_ids: (256,)}, types: {input_ids: tf.int64, input_mask: tf.int64, masked_lm_ids: tf.int64, masked_lm_positions: tf.int64, masked_lm_weights: tf.float32, next_sentence_labels: tf.int64, segment_ids: tf.int64}>

In [16]:
siz = 0
for raw_record in parsed_dataset.batch(1):
    continue